# Анализ БД сервиса для чтения книг (SQL)

## Описание проекта

### Цель исследованния

Анализ базы данных, содержащей информацию о книгах, издательствах, авторах, пользовательские обзоры книг, для формулирования ценностного предложения для нового продукта на рынке приложений и сервисов для чтения книг.

### Данные

1. Таблица *books* - данные о книгах:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.
2. Таблица *authors* - данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.
3. Таблица *publishers* - данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства.
4. Таблица *ratings* - данные о пользовательских оценках:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.
5. Таблица *reviews* - данные о пользовательских обзорах:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя автора обзора;
- `text` — текст обзора.

### Ход исследования

1. Исследовать таблиц.
2. Написать SQL-запросы:
- посчитать количество книг, вышедших после 1 января 2000 года;
- рассчитать для каждой книги количество обзоров и среднюю оценку;
- определить издательство, которое выпустило наибольшее число книг толще 50 страниц для исключения из анализ брошюр;
- определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками;
- посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.
3. Описать выводы по каждому из SQL-запросов.

## Исследование таблиц

In [3]:
#импорт библиотек
import pandas as pd
from sqlalchemy import create_engine

In [4]:
def information(df): #функция для получения общих сведений о датафрейме
    print('\033[1m' + 'Общая информация:' + '\033[0m')
    df.info()
    print()
    print('\033[1m' + 'Первые 2 строки:' + '\033[0m')
    display(df.head(2))
    print('\033[1m' + 'Количество дубликатов:' + '\033[0m', df.duplicated().sum())
    print()
    print('\033[1m' + 'Количество пропусков:' + '\033[0m')
    display(df.isna().sum())
    print('\033[1m' + 'Доля пропусков:' + '\033[0m')
    print(df.isna().sum() / len(df))

In [5]:
#установление параметров
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

#сохранение коннектора
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [6]:
query = '''SELECT *
           FROM books
        '''

In [7]:
information(pd.io.sql.read_sql(query, con = engine))

Общая информация:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB

Первые 2 строки:


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336


Количество дубликатов: 0

Количество пропусков:


book_id             0
author_id           0
title               0
num_pages           0
publication_date    0
publisher_id        0
dtype: int64

Доля пропусков:
book_id             0.0
author_id           0.0
title               0.0
num_pages           0.0
publication_date    0.0
publisher_id        0.0
dtype: float64


In [8]:
query = '''SELECT *
           FROM authors
        '''

In [9]:
information(pd.io.sql.read_sql(query, con = engine))

Общая информация:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB

Первые 2 строки:


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs


Количество дубликатов: 0

Количество пропусков:


author_id    0
author       0
dtype: int64

Доля пропусков:
author_id    0.0
author       0.0
dtype: float64


In [10]:
query = '''SELECT *
           FROM publishers
        '''

In [11]:
information(pd.io.sql.read_sql(query, con = engine))

Общая информация:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB

Первые 2 строки:


,publisher_id,publisher
0,1,Ace
1,2,Ace Book


Количество дубликатов: 0

Количество пропусков:


publisher_id    0
publisher       0
dtype: int64

Доля пропусков:
publisher_id    0.0
publisher       0.0
dtype: float64


In [12]:
query = '''SELECT *
           FROM ratings
        '''

In [13]:
information(pd.io.sql.read_sql(query, con = engine))

Общая информация:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB

Первые 2 строки:


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2


Количество дубликатов: 0

Количество пропусков:


rating_id    0
book_id      0
username     0
rating       0
dtype: int64

Доля пропусков:
rating_id    0.0
book_id      0.0
username     0.0
rating       0.0
dtype: float64


In [14]:
query = '''SELECT *
           FROM reviews
        '''

In [15]:
information(pd.io.sql.read_sql(query, con = engine))

Общая информация:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB

Первые 2 строки:


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...


Количество дубликатов: 0

Количество пропусков:


review_id    0
book_id      0
username     0
text         0
dtype: int64

Доля пропусков:
review_id    0.0
book_id      0.0
username     0.0
text         0.0
dtype: float64


### Промежуточный вывод

Информация в таблицах БД соответствует описанной в технической документации. В таблицах отсутствуют пропуски и дубликаты. 

## SQL-запросы

### Посчитать количество книг, вышедших после 1 января 2000 года

In [16]:
query = '''SELECT COUNT(book_id)
           FROM books
           WHERE publication_date > '2000-01-01'
        '''

In [17]:
pd.io.sql.read_sql(query, con = engine)

,count
0,819


В базе данных 819 книг, вышедших после 1.01.2000 года.

### Рассчитать для каждой книги количество обзоров и среднюю оценку

In [18]:
query = '''SELECT b.title,
                  COUNT(DISTINCT r.review_id) AS total_review,
                  ROUND(AVG(rat.rating), 2) AS avg_rating
           FROM books AS b LEFT JOIN reviews AS r ON b.book_id=r.book_id
                           LEFT JOIN ratings AS rat ON b.book_id=rat.book_id
           GROUP BY b.title
           ORDER BY total_review DESC, avg_rating DESC
        '''

In [19]:
pd.io.sql.read_sql(query, con = engine)

,title,total_review,avg_rating
0,Memoirs of a Geisha,8,4.14
1,Twilight (Twilight #1),7,3.66
2,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
3,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
4,The Book Thief,6,4.26
...,...,...,...
994,Disney's Beauty and the Beast (A Little Golden...,0,4.00
995,Essential Tales and Poems,0,4.00
996,Leonardo's Notebooks,0,4.00
997,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67


Больше всего обзоров у книги "Мемуары гейши" - 8, далее первая книга из серии "Сумерки" - 7 обзоров, по 6 обзоров у двух книг из серии о Гарри Поттере - "Гарри Поттер и узник Азкабана", "Гарри Поттер и Тайная комната" и у книги "Воровка книг". Для большей наглядности выведем топ-10 книг отсортируем их, как и ранее, по количеству обзоров и затем по средней оценке.

In [20]:
query = '''SELECT b.title,
                  COUNT(DISTINCT r.review_id) AS total_review,
                  ROUND(AVG(rat.rating), 2) AS avg_rating
           FROM books AS b LEFT JOIN reviews AS r ON b.book_id=r.book_id
                           LEFT JOIN ratings AS rat ON b.book_id=rat.book_id
           GROUP BY b.title
           ORDER BY total_review DESC, avg_rating DESC
           LIMIT 10
        '''

In [21]:
pd.io.sql.read_sql(query, con = engine)

,title,total_review,avg_rating
0,Memoirs of a Geisha,8,4.14
1,Twilight (Twilight #1),7,3.66
2,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
3,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
4,The Book Thief,6,4.26
5,The Glass Castle,6,4.21
6,The Hobbit or There and Back Again,6,4.13
7,Outlander (Outlander #1),6,4.13
8,The Lightning Thief (Percy Jackson and the Oly...,6,4.08
9,The Curious Incident of the Dog in the Night-Time,6,4.08


Из топ-10 книг по количеству обзоров, самой высокой средней оценкой обладает "Гарри Поттер и узник Азкобана" - 4,41, самая низкая средняя оценка у первой книги из серии "Сумерки" - 3,66.

### Определить издательство, которое выпустило наибольшее число книг толще 50 страниц для исключения из анализ брошюр

In [22]:
query = '''SELECT p.publisher,
                  COUNT(b.book_id) AS total_books
           FROM books AS b LEFT JOIN publishers AS p ON b.publisher_id=p.publisher_id
           WHERE b.num_pages > 50
           GROUP BY p.publisher
           ORDER BY total_books DESC
           LIMIT 1
        '''

In [23]:
pd.io.sql.read_sql(query, con = engine)

,publisher,total_books
0,Penguin Books,42


Наибольшее число книг, за исключение брошюр (книг толщиной 50 страниц и менее) издано в Penguin Books.

### Определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками

In [49]:
query = '''SELECT popular_books.author,
                  ROUND(AVG(r.rating), 2) AS avg_rating
           FROM
               (SELECT b.book_id,
                       a.author,
                       COUNT(r.rating)
               FROM books AS b LEFT JOIN ratings AS r ON b.book_id=r.book_id LEFT JOIN authors AS a ON b.author_id=a.author_id
               GROUP BY b.book_id, a,author
               HAVING COUNT(r.rating) >= 50) AS popular_books LEFT JOIN ratings AS r ON popular_books.book_id=r.book_id
           GROUP BY popular_books.author
           ORDER BY avg_rating DESC
           LIMIT 1
        '''

In [50]:
pd.io.sql.read_sql(query, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29


Автор с самой высокой средней оценкой книг, имеющих более 50 оценок - Джоан Роулинг.

### Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [26]:
query = '''SELECT ROUND(AVG(big_users.total_reviews)) AS avg_reviews
           FROM
               (SELECT r.username,
                      COUNT(DISTINCT review_id) AS total_reviews
               FROM reviews AS r LEFT JOIN ratings AS rat ON r.username=rat.username
               GROUP BY r.username
               HAVING COUNT(DISTINCT rating_id) > 50) AS big_users
        '''

In [27]:
pd.io.sql.read_sql(query, con = engine)

,avg_reviews
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24.

## Общий вывод

1. В базе данных 819 книг, вышедших после 1.01.2000 года.
2. Из топ-10 книг по количеству обзоров, самой высокой средней оценкой обладает "Гарри Поттер и узник Азкобана" - 4,41, самая низкая средняя оценка у первой книги из серии "Сумерки" - 3,66.
3. Наибольшее число книг, за исключение брошюр (книг толщиной 50 страниц и менее) издано в Penguin Books.
4. Автор с самой высокой средней оценкой книг, имеющих более 50 оценок - Джоан Роулинг (средняя оценка - 4,29).
5. Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24.